In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle

In [4]:
selection_strategy = "selected_5"
# read all the folders in the selection strategy
strategy_names = os.listdir(os.path.join("output", selection_strategy))

In [5]:
loop = tqdm(strategy_names)
for strategy_name in loop:  
    if "." in strategy_name:
        continue
    
    loop.set_description(f"Processing {strategy_name}")
    output_file = os.path.join("output", selection_strategy, strategy_name, "2004-01-01_2024-01-01.pkl")
    with open(output_file, "rb") as f:
        all_results = pickle.load(f)
    # level 0 keys
    rolling_windows = all_results.keys()
    # level 1 keys
    tickers = all_results[list(rolling_windows)[-1]].keys()
    
    results_df_by_tickers = pd.DataFrame(columns=["Period", "ticker", "annual_return (%)", "annual_volatility (%)", "sharpe_ratio", "sortino_ratio", "max_drawdown"])
    
    all_ticker_avg_annual_return = 0
    all_ticker_avg_annual_volatility = 0
    all_ticker_avg_sharpe_ratio = 0
    all_ticker_avg_sortino_ratio = 0
    all_ticker_avg_max_drawdown = 0
    all_ticker_valid_window = 0
    
    try:
        for ticker in tickers:
            valid_window = 0
            # calculate the average return
            avg_annual_return = 0
            avg_annual_volatility = 0
            avg_sharpe_ratio = 0
            avg_sortino_ratio = 0
            avg_max_drawdown = 0
    
            for window in rolling_windows:
                if ticker not in all_results[window]:
                    continue

                avg_annual_return += all_results[window][ticker]["annual_return"]
                avg_annual_volatility += all_results[window][ticker]["annual_volatility"]
                avg_sharpe_ratio += all_results[window][ticker]["sharpe_ratio"]
                avg_sortino_ratio += all_results[window][ticker]["sortino_ratio"]
                avg_max_drawdown += all_results[window][ticker]["max_drawdown"]

                all_ticker_avg_annual_return += all_results[window][ticker]["annual_return"]
                all_ticker_avg_annual_volatility += all_results[window][ticker]["annual_volatility"]
                all_ticker_avg_sharpe_ratio += all_results[window][ticker]["sharpe_ratio"]
                all_ticker_avg_sortino_ratio += all_results[window][ticker]["sortino_ratio"]
                all_ticker_avg_max_drawdown += all_results[window][ticker]["max_drawdown"]

                # print("="*10)
                # print(all_ticker_avg_sharpe_ratio)
                # print(all_results[window][ticker]["sharpe_ratio"])
                
                valid_window += 1
                all_ticker_valid_window += 1
                
                results_df_by_tickers = results_df_by_tickers._append(
                    {
                        "Period": window, 
                        "ticker": ticker, 
                        "annual_return (%)": round(all_results[window][ticker]["annual_return"] * 100, 4), 
                        "annual_volatility (%)": round(all_results[window][ticker]["annual_volatility"] * 100, 4),
                        "sharpe_ratio": round(all_results[window][ticker]["sharpe_ratio"], 4), 
                        "sortino_ratio": round(all_results[window][ticker]["sortino_ratio"], 4),
                        "max_drawdown": -round(all_results[window][ticker]["max_drawdown"], 4),
                     }, 
                    ignore_index=True)
        
            avg_annual_return /= valid_window
            avg_annual_volatility /= valid_window
            avg_sharpe_ratio /= valid_window
            avg_sortino_ratio /= valid_window
            avg_max_drawdown /= valid_window
            
            results_df_by_tickers = results_df_by_tickers._append(
                {
                    "Period": "Average", 
                    "ticker": ticker, 
                    "annual_return (%)": round(avg_annual_return * 100, 4), 
                    "annual_volatility (%)": round(avg_annual_volatility * 100, 4),
                    "sharpe_ratio": round(avg_sharpe_ratio, 4), 
                    "sortino_ratio": round(avg_sortino_ratio, 4),
                    "max_drawdown": -round(avg_max_drawdown, 4),
                }, 
                ignore_index=True)
            
        all_ticker_avg_annual_return /= all_ticker_valid_window
        all_ticker_avg_annual_volatility /= all_ticker_valid_window
        all_ticker_avg_sharpe_ratio /= all_ticker_valid_window
        all_ticker_avg_sortino_ratio /= all_ticker_valid_window
        all_ticker_avg_max_drawdown /= all_ticker_valid_window
        
        results_df_by_tickers = results_df_by_tickers._append(
            {
                "Period": "Average", 
                "ticker": "All", 
                "annual_return (%)": round(all_ticker_avg_annual_return * 100, 5),
                "annual_volatility (%)": round(all_ticker_avg_annual_volatility * 100, 5),
                "sharpe_ratio": round(all_ticker_avg_sharpe_ratio, 5),
                "sortino_ratio": round(all_ticker_avg_sortino_ratio, 5),
                "max_drawdown": -round(all_ticker_avg_max_drawdown, 5),
            }, 
            ignore_index=True)
        
        results_df_by_tickers.to_csv(os.path.join("output", selection_strategy, strategy_name, "results.csv"), index=False)
    except:
        print(f"Error processing {strategy_name}")
        continue

Processing BollingerBandsStrategy:   0%|          | 0/8 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'output/cherry_pick_both_finmem/BollingerBandsStrategy/2004-01-01_2024-01-01.pkl'

In [10]:
print(all_ticker_avg_sharpe_ratio / all_ticker_valid_window)

-0.0002169851882117176
